# XGBoost experiments (Michael)

## Setup

In [1]:
# import the usual suspects / basics
import time; full_run_time_start = time.time() # start timing exec right away
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy import sparse

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, f1_score,\
    accuracy_score, precision_score, recall_score, confusion_matrix

# XGBoost
from xgboost import XGBClassifier

# currently not used and thus commented out
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# display all df columns (default is 20)
pd.options.display.max_columns = None

## Utility function for testing models and tracking results

In [2]:
# empty df for storing results
test_results = pd.DataFrame(columns=['model_name',
                                'model_params',
                                'data_desc',
                                'data_size',
                                'features_no',
                                'f1',
                                'acc',
                                'recall',
                                'prec',
                                'roc_auc',
                                'cf_matrix',
                                'train_time',
                                'notes'])

def test_model(model, model_name, model_params, data_desc, X, y, notes=''):
    '''
    test_model(model, model_params, data_desc, X, y, notes='')
    
    Parameters:
    -----------
    model: instance of model to test
    model_name: name of model
    model_params: dict of (hyper)parameters passed to model
    data_desc: description of dataset (preprocessing steps etc.)
    X: feature array 
    y: target/label array
    notes: additional notes (default: empty string)
    '''

    # Split data using default of 75% for train, 25% for test.
    # Make sure test data has same toxic/nontoxic ratio as train data by
    # using stratify parameter.
    X_train, X_test, y_train, y_test =\
        train_test_split(X, y, stratify=y, random_state=42)
    
    # train model and time execution
    train_time_start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - train_time_start
    train_time_str = f'{int(train_time // 60)}m {round(train_time % 60)}s'

    # Make predictions on test set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:,1]

    return {'model_name': model_name,
            'model_params': model_params,
            'data_desc': data_desc,
            'data_size': X.shape[0],
            'features_no': X.shape[1],
            'f1': round(f1_score(y_test, y_pred), 3),
            'acc': round(accuracy_score(y_test, y_pred), 3),
            'recall': round(recall_score(y_test, y_pred), 3),
            'prec': round(precision_score(y_test, y_pred), 3),
            'roc_auc': round(roc_auc_score(y_test, y_pred_proba), 3),
            'cf_matrix': confusion_matrix(y_test, y_pred),
            'train_time': train_time_str,
            'notes': notes}

In [3]:
def store_test_result(result):
    test_results.loc[len(test_results)] = result

## Load data

In [4]:
df = pd.read_csv('data/undersampled_data_60_40.csv')
df.shape

(360835, 6)

## Optional: Create smaller sample from data to speed up experiments

In [5]:
sample_size = None

# uncomment to create sample of desired size
sample_size = 25_000

if sample_size != None:
    # ratio toxic/nontoxic
    tox_perc = 0.4
    nontox_perc = 0.6

    # number of toxic/nontoxic rows
    sample_size_tox = int(sample_size * tox_perc)
    sample_size_nontox = int(sample_size * nontox_perc)

    sample_tox = df[df['toxic'] == 1].sample(sample_size_tox,
                                             random_state=42)
    sample_nontox = df[df['toxic'] == 0].sample(sample_size_nontox,
                                                random_state=42)

    df = pd.concat([sample_tox, sample_nontox])
    print(f'Using sample ({df.shape[0]} rows).')

else:
    print(f'Using full data ({df.shape[0]} rows).')

Using sample (25000 rows).


## Drop rows with NaN's

In [6]:
rows_before = df.shape[0]
print("rows with NaN's before dropping:", df.shape[0])
df.dropna(inplace=True)
print('rows after:', df.shape[0])
print('rows dropped:', rows_before - df.shape[0])

rows with NaN's before dropping: 25000
rows after: 24957
rows dropped: 43


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24957 entries, 1477 to 3711
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   comment_text           24957 non-null  object
 1   toxic                  24957 non-null  int64 
 2   stopwords_punct_lemma  24957 non-null  object
 3   vector_spacy           24957 non-null  object
 4   pos_tags               24957 non-null  object
 5   pos_tags_str           24957 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB


In [8]:
df.head()

,comment_text,toxic,stopwords_punct_lemma,vector_spacy,pos_tags,pos_tags_str
1477,Good. Let's hope the president listens to his...,1,good let hope president listen adviser inste...,[ 1.4473382 0.9890728 -2.8375092 0.658990...,"[('Good', 'JJ'), ('.', '.'), ('Let', 'VB'), (""...","JJ . VB POS VB DT NN VBZ TO PRP$ NNS RB , RB I..."
67285,Actually I'd consider people like Lindsey cons...,1,actually consider people like Lindsey conserva...,[-6.41708255e-01 5.59695244e-01 -1.90619397e+...,"[('Actually', 'RB'), ('I', 'PRP'), (""'d"", 'MD'...","RB PRP MD VB NNS IN NNP JJ , IN DT PRP VBP , R..."
98,Once again the left proves they are the scum o...,1,left prove scum Earth \n\n win election spend ...,[-2.0883 -0.19458179 -0.6892491 1.014075...,"[('Once', 'RB'), ('again', 'RB'), ('the', 'DT'...",RB RB DT NN VBZ PRP VBP DT NN IN DT NN . PRP M...
269922,And Trump should be fired if his tweets are in...,1,Trump fire tweet insensitive childish demente,[ 4.85373288e-01 2.90858328e-01 -1.57731009e+...,"[('And', 'CC'), ('Trump', 'NNP'), ('should', '...","CC NNP MD VB VBN IN PRP$ NNS VBP JJ , JJ , CC ..."
80321,Making decisions based on factors as nebulous ...,1,make decision base factor nebulous Paris Accor...,[-0.34122247 0.5101946 -2.1777446 0.028234...,"[('Making', 'VBG'), ('decisions', 'NNS'), ('ba...",VBG NNS VBN IN NNS RB JJ IN DT NNP NNP VBZ DT ...


## Create label/target variable and check for imbalance

In [9]:
target = df['toxic']

In [10]:
value_counts = target.value_counts()
nontoxic_count = value_counts[0]
toxic_count = value_counts[1]
nontoxic_perc =\
    round((nontoxic_count / (nontoxic_count + toxic_count)) * 100, 1)
toxic_perc =\
    round((toxic_count / (nontoxic_count + toxic_count)) * 100, 1)

print(f'Nontoxic (0): {nontoxic_count} ({nontoxic_perc} %)')
print(f'Toxic (1): {toxic_count} ({toxic_perc} %)')

Nontoxic (0): 14957 (59.9 %)
Toxic (1): 10000 (40.1 %)


## Create various corpora

### Raw corpus

In [11]:
corp_raw = df['comment_text']
corp_raw.shape

(24957,)

### Pre-processed corpus

In [12]:
corp_pp = df['stopwords_punct_lemma']
corp_pp.shape

(24957,)

### Corpus of spacy vectors

In [13]:
# If smaller sample: Convert vector string in csv file to df
# and cast all cols as float. This takes ~50 min for the full 360,000 rows.
# --> If full data: Load pickle file to save time.

if sample_size != None:
    corp_spacy = df['vector_spacy'].str.strip('[]').str.split(expand=True)
    corp_spacy = corp_spacy.astype('float')
    display(corp_spacy)
    # with open('pickle/spacy_vectors.pkl', mode='wb') as f:
    #     pickle.dump(corp_spacy, f)

else:
    with open('pickle/spacy_vectors.pkl', mode='rb') as f:
        corp_spacy = pickle.load(f)
    display(corp_spacy)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
1477,1.447338,0.989073,-2.837509,0.658991,0.672710,1.191676,0.285207,2.426576,-1.692114,-0.176049,3.934427,1.150893,-2.356241,0.828215,-0.992825,0.729421,1.706436,-1.819391,0.097194,0.431922,0.255678,-1.266474,-0.450235,-2.383436,0.009307,-0.240100,-1.485890,-0.873162,-0.050700,1.801850,0.228427,-1.292545,0.655094,-0.801870,-0.541164,0.219838,-1.053595,0.318381,1.679943,2.140065,-1.655223,0.311242,1.078556,-0.442303,-0.744830,0.782592,1.409505,-3.122552,-0.567920,1.365632,-1.327211,0.018819,0.639722,-2.035966,0.039106,-0.378086,-0.113403,1.566775,0.674308,0.935797,2.073059,-0.678601,1.153684,-1.253865,0.296068,1.032703,-1.863666,-3.160026,0.633379,0.949073,2.227246,-0.222558,-0.692850,-0.598044,0.661155,1.116506,-1.085037,1.812174,0.102856,-2.265764,-2.748927,-2.635322,1.254545,0.473484,-0.894685,0.560620,0.064153,-2.546222,1.218625,-0.985504,-0.081563,1.232141,1.038742,-2.360090,1.366507,-2.262342,0.024480,0.120594,0.356569,0.199665,1.181463,1.211473,0.240170,1.895987,-1.337040,0.415248,-2.096530,-0.994066,-0.162791,-1.933663,0.716135,-1.745517,-0.585685,-1.577185,-0.744824,2.713183,-3.170229,-1.943473,0.947423,0.017556,-0.923856,-1.776971,-1.686727,0.173288,-0.638135,-0.346419,3.243218,-0.827596,1.880518,-1.020007,-1.331072,-0.085317,1.712556,-0.242993,-1.069617,-0.945440,-1.918320,-0.973944,2.510781,-1.680616,-2.636132,-0.691855,0.231941,1.484493,-1.024885,-1.107356,-2.590645,0.679043,-0.559594,-0.519887,-1.352685,0.725248,0.410003,1.080092,-1.988682,2.303114,3.084788,0.310003,-0.553435,-2.390043,0.224718,-1.740911,-1.353859,-0.130484,-0.110978,-0.661164,-0.839274,-0.202948,-0.819933,-1.205320,0.651909,0.534960,1.434445,-0.481054,0.601297,-0.603026,-1.133857,0.355430,-0.495364,-0.707020,1.155727,0.389672,2.976284,-0.203934,0.632264,0.594518,1.606209,-2.612810,1.469768,2.093344,1.044416,1.166731,-0.741649,0.625016,-3.061623,-1.348242,-2.119584,0.506404,-0.549953,1.813935,-1.955853,-0.183054,-0.938769,-1.985775,-2.123790,-0.154973,0.126572,0.387217,-0.911598,0.729371,0.330887,0.848217,-0.450927,-0.131697,0.103526,2.735063,-0.264879,-1.151768,0.509372,-0.359525,-1.330164,0.532508,0.564131,0.786888,-0.064036,-0.620366,-0.116083,1.616256,1.487243,1.326860,-0.664096,-3.354927,-1.936418,-1.481669,-0.025682,-0.459806,-1.747816,-0.333415,0.532155,-0.980654,0.216045,0.187075,2.790808,-0.903923,-0.485790,1.244636,-0.749542,-0.568840,0.947629,1.091582,0.024955,-1.113088,-3.488241,-0.628379,1.385689,-2.668185,0.719956,-1.036315,-0.729682,1.257612,-1.230474,1.091475,1.178266,0.170564,2.185019,0.281890,0.478311,0.386657,-2.397473,-0.324701,0.633647,0.588723,-0.340915,-1.158466,-0.022949,-1.624240,2.472658,-2.356169,-1.306742,2.624767,-1.245684,0.180542,1.080140,0.618075,1.281191,0.733621,1.017922,0.498706,-2.452836,-0.123139,0.440162,-1.283612,-0.277114,-0.510127,-0.449223,-0.259792,-0.252739,1.534381,-2.858636,1.419309
67285,-0.641708,0.559695,-1.906194,-0.129815,0.92

### Bag of words (default)

In [14]:
vect_bow = CountVectorizer()
corp_bow = vect_bow.fit_transform(corp_raw)
corp_bow

<24957x41168 sparse matrix of type '<class 'numpy.int64'>'
	with 954037 stored elements in Compressed Sparse Row format>

In [15]:
# output just a small number of features, else kernel crashes while converting
# sparse matrix to array
n_words = 100
pd.DataFrame(data=corp_bow[:, 10000:10000+n_words].toarray(),
             columns=vect_bow.get_feature_names_out()[10000:10000+n_words])

,defenceman,defences,defend,defendant,defendants,defended,defender,defenders,defending,defends,defense,defenseless,defenseman,defensemen,defenses,defensive,defensively,defer,deference,deferential,defermements,deferment,deferments,deferral,deferrals,deferred,deferred_action_for_childhood_arrivals,deferring,defiance,defiant,deficiences,deficiencies,deficiency,deficient,deficit,deficits,defied,defies,defile,define,defined,defines,defining,definite,definitely,definition,definitions,definitive,definitively,deflated,deflationary,deflect,deflected,deflecting,deflection,deflections,deflects,deformed,defraud,defrauded,defrauding,defrauds,defrock,defrocked,defrocking,defrocks,defunct,defund,defunded,defunding,defuse,defy,defying,degeneracy,degenerate,degenerated,degenerates,degenerating,degenerative,degette,degettes,degollado,degradation,degrade,degraded,degrading,degrassi,degree,degreed,degrees,dehavilland,dehumanize,dehumanized,dehumanizes,dehumanizing,dehydration,dei,deified,deigned,deity
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24952,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24955,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Bag of words (binary)

In [16]:
vect_bow_bin = CountVectorizer(binary=True)
corp_bow_bin = vect_bow_bin.fit_transform(corp_raw)
corp_bow_bin

<24957x41168 sparse matrix of type '<class 'numpy.int64'>'
	with 954037 stored elements in Compressed Sparse Row format>

### Bag of words (mixed case)

In [17]:
vect_bow_mixc = CountVectorizer(lowercase=False)
corp_bow_mixc = vect_bow_mixc.fit_transform(corp_raw)
corp_bow_mixc

<24957x51449 sparse matrix of type '<class 'numpy.int64'>'
	with 977589 stored elements in Compressed Sparse Row format>

### Bag of words (default) on preprocessed comments (lemmatization, stopword and punctuation removal)

In [18]:
vect_bow = CountVectorizer()
corp_pp_bow = vect_bow.fit_transform(corp_pp)
corp_pp_bow

<24957x33999 sparse matrix of type '<class 'numpy.int64'>'
	with 516347 stored elements in Compressed Sparse Row format>

### Bag of 1/2-grams (default) on preprocessed comments

In [19]:
vect_bo12grams = CountVectorizer(ngram_range=(1,2))
corp_pp_bo12grams = vect_bo12grams.fit_transform(corp_pp)
corp_pp_bo12grams

<24957x464367 sparse matrix of type '<class 'numpy.int64'>'
	with 1067013 stored elements in Compressed Sparse Row format>

### Bag of 1/2/3-grams (default) on preprocessed comments

In [20]:
vect_bo123grams = CountVectorizer(ngram_range=(1,3))
corp_pp_bo123grams = vect_bo123grams.fit_transform(corp_pp)
corp_pp_bo123grams

<24957x989750 sparse matrix of type '<class 'numpy.int64'>'
	with 1600456 stored elements in Compressed Sparse Row format>

### Bag of 2-grams (default) on preprocessed comments

In [21]:
vect_bo2grams = CountVectorizer(ngram_range=(2,2))
corp_pp_bo2grams = vect_bo2grams.fit_transform(corp_pp)
corp_pp_bo2grams

<24957x430368 sparse matrix of type '<class 'numpy.int64'>'
	with 550666 stored elements in Compressed Sparse Row format>

### Tf_idf

In [22]:
vect_tfidf = TfidfVectorizer()
corp_tfidf = vect_tfidf.fit_transform(corp_raw)
corp_tfidf

<24957x41168 sparse matrix of type '<class 'numpy.float64'>'
	with 954037 stored elements in Compressed Sparse Row format>

In [23]:
# output just a small number of features, else kernel crashes
n_words = 100
pd.DataFrame(data=corp_tfidf[:, 10000:10000+n_words].toarray(),
             columns=vect_tfidf.get_feature_names_out()[10000:10000+n_words])

,defenceman,defences,defend,defendant,defendants,defended,defender,defenders,defending,defends,defense,defenseless,defenseman,defensemen,defenses,defensive,defensively,defer,deference,deferential,defermements,deferment,deferments,deferral,deferrals,deferred,deferred_action_for_childhood_arrivals,deferring,defiance,defiant,deficiences,deficiencies,deficiency,deficient,deficit,deficits,defied,defies,defile,define,defined,defines,defining,definite,definitely,definition,definitions,definitive,definitively,deflated,deflationary,deflect,deflected,deflecting,deflection,deflections,deflects,deformed,defraud,defrauded,defrauding,defrauds,defrock,defrocked,defrocking,defrocks,defunct,defund,defunded,defunding,defuse,defy,defying,degeneracy,degenerate,degenerated,degenerates,degenerating,degenerative,degette,degettes,degollado,degradation,degrade,degraded,degrading,degrassi,degree,degreed,degrees,dehavilland,dehumanize,dehumanized,dehumanizes,dehumanizing,dehydration,dei,deified,deigned,deity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Tf_idf on preprocessed comments (lemmatization, stopword and punctuation removal)

In [24]:
vect_tfidf = TfidfVectorizer()
corp_pp_tfidf = vect_tfidf.fit_transform(corp_pp)
corp_pp_tfidf

<24957x33999 sparse matrix of type '<class 'numpy.float64'>'
	with 516347 stored elements in Compressed Sparse Row format>

## Baseline model (logistic regression)

In [25]:
# parameters for model
params = {'max_iter': 2_000}

# load model with parameters
lr = LogisticRegression(**params)

test_result = test_model(lr, 'BASELINE (logistic regression)', params,
                    'bag of words', corp_bow, target)
store_test_result(test_result)

## XGBoost experiments

In [26]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'bag of words',
                         corp_bow, target)
store_test_result(test_result)

In [27]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'bag of words (binary)',
                         corp_bow_bin, target)
store_test_result(test_result)

In [28]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'bag of words (mixed case)',
                         corp_bow_mixc, target)
store_test_result(test_result)

In [29]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'bag of words (preprocessed)',
                         corp_pp_bow, target)
store_test_result(test_result)

In [30]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params,
                         'bag of 1/2-grams (preprocessed)',
                         corp_pp_bo12grams, target)
store_test_result(test_result)

In [31]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params,
                         'bag of 1/2/3-grams (preprocessed)',
                         corp_pp_bo123grams, target)
store_test_result(test_result)

In [32]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params,
                         'bag of 2-grams (preprocessed)',
                         corp_pp_bo2grams, target)
store_test_result(test_result)

In [33]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'tf_idf',
                         corp_tfidf, target)
store_test_result(test_result)

In [34]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'tf_idf (preprocessed)',
                         corp_pp_tfidf, target)
store_test_result(test_result)

In [35]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1,
          'n_estimators': 1000}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'tf_idf (preprocessed)',
                         corp_pp_tfidf, target)
store_test_result(test_result)

In [36]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'spacy vectors (300-D)',
                         corp_spacy, target)
store_test_result(test_result)

In [37]:
# parameters for model
params = {'random_state': 42,
          'n_jobs': -1,
          'n_estimators': 1000}

# load model with parameters
xgb = XGBClassifier(**params)

test_result = test_model(xgb, 'XGBoost', params, 'spacy vectors (300-D)',
                         corp_spacy, target)
store_test_result(test_result)

## Show test results + total exec time

In [38]:
test_results

,model_name,model_params,data_desc,train_data_size,features_no,f1,acc,recall,prec,roc_auc,cf_matrix,train_time,notes
0,BASELINE (logistic regression),{'max_iter': 2000},bag of words,18717,41168,0.777,0.832,0.728,0.832,0.894,"[[3373, 367], [679, 1821]]",0m 2s,
1,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of words,18717,41168,0.756,0.830,0.655,0.893,0.897,"[[3544, 196], [862, 1638]]",0m 1s,
2,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of words (binary),18717,41168,0.747,0.825,0.646,0.885,0.898,"[[3530, 210], [884, 1616]]",0m 1s,
3,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of words (mixed case),18717,51449,0.733,0.818,0.626,0.885,0.885,"[[3536, 204], [934, 1566]]",0m 1s,
4,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of words (preprocessed),18717,33999,0.758,0.829,0.669,0.874,0.899,"[[3500, 240], [828, 1672]]",0m 1s,
5,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of 1/2-grams (preprocessed),18717,464367,0.760,0.831,0.668,0.883,0.900,"[[3518, 222], [830, 1670]]",0m 6s,
6,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of 1/2/3-grams (preprocessed),18717,989750,0.760,0.831,0.668,0.883,0.900,"[[3518, 222], [830, 1670]]",0m 12s,
7,XGBoost,"{'random_state': 42, 'n_jobs': -1}",bag of 2-grams (preprocessed),18717,430368,0.251,0.641,0.150,0.763,0.595,"[[3623, 117], [2124, 376]]",0m 3s,
8,XGBoost,"{'random_state': 42, 'n_jobs': -1}",tf_idf,18717,41168,0.754,0.827,0.659,0.880,0.892,"[[3516, 224], [853, 1647]]",0m 5s,
9,XGBoost,"{'random_state': 42, 'n_jobs': -1}",tf_idf (preprocessed),18717,33999,0.768,0.835,0.681,0.880,0.902,"[[3508, 232], [798, 1702]]",0m 4s,


In [39]:
full_run_time = time.time() - full_run_time_start
print(f'Full run time: {int(full_run_time // 60)}m {round(full_run_time % 60)}s')

Full run time: 1m 56s


## Notes

- also try LightGBM?